# Cut motion and physiology measurements per piece performed

This notebook takes full concert measurement csv files for the Stavangar Lydo 2023 performances and outputs excerpt csvs for each piece.

In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json 
from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate
from scipy.interpolate import interp1d

In [2]:
%load_ext autoreload
%autoreload 1
%aimport eq
%aimport qex
%aimport al

In [3]:
%reload_ext autoreload

In [11]:
Piece_timings = pd.read_csv('./Timing/ctimes.csv',index_col = 0)
concerts = list(Piece_timings.columns[1:])
print(Piece_timings.index)
Piece_timings.iloc[:5,:]


Index(['Pre-Audio', 'Warmup', 'Tuning', 'Synch Taps', 'Applause for Conductor',
       'Saevarud', 'Applause for Saevarud', 'MC pre Reich', 'Reich',
       'Applause for Reich', 'MC pre Arban', 'Applause for Florian', 'Arban',
       'Applause for Arban', 'MC pre Grieg', 'Grieg', 'Applause for Grieg',
       'MC pre Tsjajkovskij', 'Tsjajkovskij', 'Applause for Tsjajkovskij',
       'MC pre Christophersen', 'Christophersen',
       'Applause for Christophersen', 'MC pre School',
       'Applause for school class', 'MC pre Vivaldi', 'Vivaldi',
       'Applause for Vivaldi', 'MC post Vivaldi', 'Applause for school class',
       'MC pre Strauss', 'Strauss', 'Applause for Strauss', 'End'],
      dtype='object', name='Event')


,RD,C1,C2,C3,C4,C5
Event,,,,,,
Pre-Audio,NaN,NaN,-421.378,NaN,NaN,-981.902
Warmup,-300.000,-348.482,-361.303,-52.946,-344.999,-386.462
Tuning,-58.745,-61.511,-49.122,-48.773,-48.983,-225.636
Synch Taps,0.000,0.000,0.000,0.000,0.000,0.000
Applause for Conductor,19.294,28.292,122.014,76.074,21.929,-30.137


In [20]:
Show_folders = {
    'RD':'../Data/Equivital/Aligned/20230215_RD/Full/',
    'C1':'../Data/Equivital/Aligned/20230216_C1/Full/',
    'C2':'../Data/Equivital/Aligned/20230216_C2/Full/',
    'C3':'../Data/Equivital/Aligned/20230217_C3/Full/',
    'C4':'../Data/Equivital/Aligned/20230217_C4/Full/',
    'C5':'../Data/Equivital/Aligned/20230218_C5/Full/'}
concerts = list(Show_folders.keys())
Piece_folders = {
    'RD':'../Data/Equivital/Aligned/20230215_RD/Pieces/',
    'C1':'../Data/Equivital/Aligned/20230216_C1/Pieces/',
    'C2':'../Data/Equivital/Aligned/20230216_C2/Pieces/',
    'C3':'../Data/Equivital/Aligned/20230217_C3/Pieces/',
    'C4':'../Data/Equivital/Aligned/20230217_C4/Pieces/',
    'C5':'../Data/Equivital/Aligned/20230218_C5/Pieces/'}
# Saev - Sævarud, Reic - Reich, Arba - Arban, Tcha - Tsjakovskij, Chri - Christophersen, Viva - Vivaldi-Vinje, Stra - Strauss 
Piece_timemap = {'Saev':['Saevarud', 'Applause for Saevarud'],
                 'Reic':['Reich','Applause for Reich'],
                 'Arba':['Arban','Applause for Arban'],
                 'Tcha':['Tsjajkovskij', 'Applause for Tsjajkovskij'],
                 'Chri':['Christophersen','Applause for Christophersen'],
                 'Viva':['Vivaldi','Applause for Vivaldi'],
                 'Stra':['Strauss', 'Applause for Strauss']}
Pieces = list(Piece_timemap.keys())
Full_composers = {'Saev':'Sævarud', 'Reic':'Reich', 'Arba':'Arban', 'Tcha':'Tsjakovskij', 'Chri':'Christophersen', 'Viva':'Vivaldi-Vinje', 'Stra':'Strauss'}
datafile_types = ['EQDATA','EQCIBI','EQRESP','EQBACC','EQ2ECG']

This notebook takes concert time aligned measurements and cuts them down to piece wise intervals, save within piece specific folders

In [27]:
# across concerts
concert = concerts[1]
# across datafile types
dft = datafile_types[0]
buffer_s = 15 # seconds before and after each piece is included in every excerpt

data_loc = Show_folders[concert]
files = os.listdir(data_loc)
files.sort()
sig_files = {}
for f in files: 
    if f.endswith(dft+'.csv'): 
        partID = f.split('_')[2]
        sig_files[partID] = data_loc+f
participants = list(sig_files.keys())

# across participants
partID = participants[0]
V = pd.read_csv(sig_files[partID])
# across pieces
pieceID = Pieces[0]
pZero = Piece_timings.loc[Piece_timemap[pieceID][0],concert]
pEnd = Piece_timings.loc[Piece_timemap[pieceID][1],concert]


X = V.loc[V['c_sTime']>pZero - buffer_s,:].copy()
XX = X.loc[X['c_sTime']<pEnd + buffer_s,:].copy().reset_index(drop=True)
cols = XX.columns
signal_segment = pd.DataFrame()
signal_segment['p_sTime'] = XX['c_sTime']-pZero
signal_segment.loc[:,cols] = XX

filename = '_'.join([concert,pieceID,partID,dft]) + '.csv'
filepath = Piece_folders[concert]+'/'.join([pieceID,dft,''])

# check if the folder exists for project and set up locations
if not os.path.isdir(Piece_folders[concert] + pieceID):
    os.mkdir(Piece_folders[concert] + pieceID)
if not os.path.isdir(Piece_folders[concert] + pieceID + '/' + dft):
    os.mkdir(Piece_folders[concert] + pieceID + '/' + dft)

signal_segment.to_csv(filepath + filename)
os.listdir(filepath)

['C1_Saev_BR601_EQDATA.csv']

In [ ]:
buffer_s = 15 # seconds before and after each piece is included in every excerpt

In [29]:
# across concerts
toc = time.time()
for concert in concerts:
    tic = time.time()
    data_loc = Show_folders[concert]
    files = os.listdir(data_loc)
    files.sort()

    # across datafile types
    for dft in datafile_types:
        sig_files = {}
        for f in files: 
            if f.endswith(dft+'.csv'): 
                partID = f.split('_')[2]
                sig_files[partID] = data_loc+f
        participants = list(sig_files.keys())

        # across participants
        for partID in participants:
            V = pd.read_csv(sig_files[partID])
            
            # across pieces
            for pieceID in Pieces:
                pZero = Piece_timings.loc[Piece_timemap[pieceID][0],concert]
                pEnd = Piece_timings.loc[Piece_timemap[pieceID][1],concert]


                X = V.loc[V['c_sTime']>pZero - buffer_s,:].copy()
                XX = X.loc[X['c_sTime']<pEnd + buffer_s,:].copy().reset_index(drop=True)
                cols = XX.columns
                signal_segment = pd.DataFrame()
                signal_segment['p_sTime'] = np.round(XX['c_sTime']-pZero,3)
                signal_segment.loc[:,cols] = XX

                filename = '_'.join([concert,pieceID,partID,dft]) + '.csv'
                filepath = Piece_folders[concert]+'/'.join([pieceID,dft,''])

                # check if the folder exists for project and set up locations
                if not os.path.isdir(Piece_folders[concert] + pieceID):
                    os.mkdir(Piece_folders[concert] + pieceID)
                if not os.path.isdir(Piece_folders[concert] + pieceID + '/' + dft):
                    os.mkdir(Piece_folders[concert] + pieceID + '/' + dft)

                signal_segment.to_csv(filepath + filename)
                #os.listdir(filepath)
                
        print(' '.join([concert,dft,str(np.round(time.time() - tic,3)),str(np.round(time.time() - toc,3))]))


RD EQDATA 2.852 2.852
RD EQCIBI 5.669 5.67
RD EQRESP 26.619 26.62
RD EQBACC 251.282 251.282
RD EQ2ECG 251.282 251.282
C1 EQDATA 3.039 254.321
C1 EQCIBI 6.148 257.43
C1 EQRESP 29.113 280.395
C1 EQBACC 274.513 525.796
C1 EQ2ECG 279.063 530.346
C2 EQDATA 2.922 533.267
C2 EQCIBI 5.892 536.238
C2 EQRESP 27.895 558.241
C2 EQBACC 281.423 811.769
C2 EQ2ECG 286.167 816.513
C3 EQDATA 2.859 819.372
C3 EQCIBI 5.892 822.405
C3 EQRESP 29.973 846.486
C3 EQBACC 254.043 1070.556
C3 EQ2ECG 490.937 1307.45
C4 EQDATA 2.831 1310.281
C4 EQCIBI 5.539 1312.989
C4 EQRESP 25.392 1332.842
C4 EQBACC 243.195 1550.645
C4 EQ2ECG 472.497 1779.947
C5 EQDATA 2.705 1782.653
C5 EQCIBI 5.523 1785.47
C5 EQRESP 25.641 1805.588
C5 EQBACC 262.221 2042.168
C5 EQ2ECG 568.424 2348.371


In [34]:
Show_folders = {
    'RD':'../Data/AX3/Aligned/20230215_RD/Full/',
    'C1':'../Data/AX3/Aligned/20230216_C1/Full/',
    'C2':'../Data/AX3/Aligned/20230216_C2/Full/',
    'C3':'../Data/AX3/Aligned/20230217_C3/Full/',
    'C4':'../Data/AX3/Aligned/20230217_C4/Full/',
    'C5':'../Data/AX3/Aligned/20230218_C5/Full/'}
concerts = list(Show_folders.keys())
Piece_folders = {
    'RD':'../Data/AX3/Aligned/20230215_RD/Pieces/',
    'C1':'../Data/AX3/Aligned/20230216_C1/Pieces/',
    'C2':'../Data/AX3/Aligned/20230216_C2/Pieces/',
    'C3':'../Data/AX3/Aligned/20230217_C3/Pieces/',
    'C4':'../Data/AX3/Aligned/20230217_C4/Pieces/',
    'C5':'../Data/AX3/Aligned/20230218_C5/Pieces/'}
# Saev - Sævarud, Reic - Reich, Arba - Arban, Tcha - Tsjakovskij, Chri - Christophersen, Viva - Vivaldi-Vinje, Stra - Strauss 
Piece_timemap = {'Saev':['Saevarud', 'Applause for Saevarud'],
                 'Reic':['Reich','Applause for Reich'],
                 'Arba':['Arban','Applause for Arban'],
                 'Tcha':['Tsjajkovskij', 'Applause for Tsjajkovskij'],
                 'Chri':['Christophersen','Applause for Christophersen'],
                 'Viva':['Vivaldi','Applause for Vivaldi'],
                 'Stra':['Strauss', 'Applause for Strauss']}
Pieces = list(Piece_timemap.keys())
Full_composers = {'Saev':'Sævarud', 'Reic':'Reich', 'Arba':'Arban', 'Tcha':'Tsjakovskij', 'Chri':'Christophersen', 'Viva':'Vivaldi-Vinje', 'Stra':'Strauss'}

In [37]:
# across concerts
concert = concerts[1]
# across datafile types
buffer_s = 15 # seconds before and after each piece is included in every excerpt

data_loc = Show_folders[concert]
files = os.listdir(data_loc)
dft = 'AX3ACC'
files.sort()
sig_files = {}
for f in files: 
    if f.endswith(dft+'.csv'): 
        partID = f.split('_')[2]
        sig_files[partID] = data_loc+f
participants = list(sig_files.keys())

# across participants
partID = participants[0]
V = pd.read_csv(sig_files[partID])
# across pieces
pieceID = Pieces[0]
pZero = Piece_timings.loc[Piece_timemap[pieceID][0],concert]
pEnd = Piece_timings.loc[Piece_timemap[pieceID][1],concert]


X = V.loc[V['c_sTime']>pZero - buffer_s,:].copy()
XX = X.loc[X['c_sTime']<pEnd + buffer_s,:].copy().reset_index(drop=True)
cols = XX.columns
signal_segment = pd.DataFrame()
signal_segment['p_sTime'] = XX['c_sTime']-pZero
signal_segment.loc[:,cols] = XX

filename = '_'.join([concert,pieceID,partID,dft]) + '.csv'
filepath = Piece_folders[concert]+'/'.join([pieceID,dft,''])

# check if the folder exists for project and set up locations
if not os.path.isdir(Piece_folders[concert] + pieceID):
    os.mkdir(Piece_folders[concert] + pieceID)
if not os.path.isdir(Piece_folders[concert] + pieceID + '/' + dft):
    os.mkdir(Piece_folders[concert] + pieceID + '/' + dft)

signal_segment.to_csv(filepath + filename)
os.listdir(filepath)

['C1_Saev_LS401_AX3ACC.csv']

In [38]:
# across concerts
toc = time.time()
for concert in concerts:
    tic = time.time()
    data_loc = Show_folders[concert]
    files = os.listdir(data_loc)
    files.sort()


    sig_files = {}
    for f in files: 
        if f.endswith(dft+'.csv'): 
            partID = f.split('_')[2]
            sig_files[partID] = data_loc+f
    participants = list(sig_files.keys())

    # across participants
    for partID in participants:
        V = pd.read_csv(sig_files[partID])

        # across pieces
        for pieceID in Pieces:
            pZero = Piece_timings.loc[Piece_timemap[pieceID][0],concert]
            pEnd = Piece_timings.loc[Piece_timemap[pieceID][1],concert]


            X = V.loc[V['c_sTime']>pZero - buffer_s,:].copy()
            XX = X.loc[X['c_sTime']<pEnd + buffer_s,:].copy().reset_index(drop=True)
            cols = XX.columns
            signal_segment = pd.DataFrame()
            signal_segment['p_sTime'] = np.round(XX['c_sTime']-pZero,3)
            signal_segment.loc[:,cols] = XX

            filename = '_'.join([concert,pieceID,partID,dft]) + '.csv'
            filepath = Piece_folders[concert]+'/'.join([pieceID,dft,''])

            # check if the folder exists for project and set up locations
            if not os.path.isdir(Piece_folders[concert] + pieceID):
                os.mkdir(Piece_folders[concert] + pieceID)
            if not os.path.isdir(Piece_folders[concert] + pieceID + '/' + dft):
                os.mkdir(Piece_folders[concert] + pieceID + '/' + dft)

            signal_segment.to_csv(filepath + filename)
            #os.listdir(filepath)

    print(' '.join([concert,dft,str(np.round(time.time() - tic,3)),str(np.round(time.time() - toc,3))]))


RD AX3ACC 32.635 32.636
C1 AX3ACC 37.646 70.282
C2 AX3ACC 34.873 105.155
C3 AX3ACC 34.318 139.473
C4 AX3ACC 33.444 172.918
C5 AX3ACC 33.302 206.22


The files produced by this alignment process include four sets of timestamps

1. p_stime: In seconds, the time from the start of this piece, with zero being aligned with the onset of the first audible note in the audio recordings fo the performance. This series starts consistently at -15 s, or 15 seconds before the music starts, to capture the interval of time in which the musicians and conductor prepared to start playing.
2. c_stime: In seconds, the time from this concert's synchronisation cue, with zero marking the time at which the musicians' first tap should align with the sync cue beeps. Note that the alignment procedure compensates for many performance inconsistencies and deviations. Not all ACC measurements have a "tap" at concert time zero, and other cue times may have been used to align device recordings.
3. rec_dtime: In a datetime string, the UTC time of each time stamp, after alignment
4. dev_dtime: In its original datetime format, the timestamp provided by the datalogging device. These values can be off by as much as 6 seconds. 

Otherwise, the sensor values are preserved as initial recorded. Not signals have been smooth, filtered, normalised. 